In [ ]:
# download the source code
!git clone https://github.com/Museum7432/college_assignments.git

%cd college_assignments/IR_CS336/factcheck_vi

# install dependencies

In [ ]:
!sudo apt install default-jdk
%pip install -r requirements.txt

# download data and checkpoint

In [ ]:
!mkdir documents encoded checkpoints indexes

# download documents
!gdown 1QybiXQ1uQkWowCo5PgTUHMtXHQmEfsVx

!unzip -d documents articles_text.jsonl.zip

# download checkpoint

!gdown 1LHHE_fwMbr4FNx1YiJIysFl9Ezx6iBd3
!mv 22311.ckpt checkpoints

# index the corpus with pyserini

In [13]:
# encode documents into vectors using 'bkai-foundation-models/vietnamese-bi-encoder'
!python -m pyserini.encode \
    input   --corpus documents \
            --fields text \
            --delimiter "^" \
    output  --embeddings="encoded" \
            --to-faiss \
    encoder --encoder bkai-foundation-models/vietnamese-bi-encoder \
            --fields text \
            --batch 4 \
            --fp16

6220it [00:00, 40278.23it/s]
 50%|████████████████████                    | 782/1555 [01:53<01:52,  6.90it/s]
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/arch/.pyenv/versions/factcheck_vi/lib/python3.11/site-packages/pyserini/encode/__main__.py", line 154, in <module>
    embeddings = encoder.encode(**kwargs)
                 ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arch/.pyenv/versions/factcheck_vi/lib/python3.11/site-packages/pyserini/encode/_auto.py", line 59, in encode
    outputs = self.model(**inputs)
              ^^^^^^^^^^^^^^^^^^^^
  File "/home/arch/.pyenv/versions/factcheck_vi/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/arch/.pyenv/versions/factcheck_vi/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1527, 

In [1]:
# then index those vector using faiss
!python -m pyserini.index.faiss \
  --input encoded \
  --output indexes/vnexpress \
  --hnsw \
  --pq

# we use lucence here mainly to quickly access the document using its id 
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input documents \
  --index indexes/raw \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

(3128, 768)
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3128 points to 256 centroids:

# start streamlit server

In [1]:
!streamlit run Entry.py \
    --server.port 8501 \
    --server.headless true \
    --client.toolbarMode minimal \
    --server.fileWatcherType none \
    -- \
    --checkpoint_path checkpoints/22311.ckpt \
    --index_dir indexes/vnexpress \
    --raw_doc_index indexes/raw \
    --cuda \
    --batch_size 2


  You can now view your Streamlit app in your browser.

  Network URL: http://172.16.168.177:8501
  External URL: http://146.70.160.174:8501

Lightning automatically upgraded your loaded checkpoint from v1.2.1 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint checkpoints/22311.ckpt`
You are using a model of type roberta to instantiate a model of type longformer. This is not supported for all configurations of models and can yield errors.
/home/arch/.pyenv/versions/factcheck_vi/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.embeddings.position_ids']
Lee Sun Kyun dùng ma túy
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/arch/Projects/college/Assignments/IR_CS336/factcheck_vi/./src/allennlp_nn_util.py:1447: UserWarning: